# Auto Calibration & Poses (No UI)

### What it does
1. Clone **aissitt/CSE5283_calibration**.
2. Run `scripts/run_calibration.py` on a folder of checkerboard images.
3. Generate corner visualizations and **axes overlays**.
4. Parse `data/results/calibration.json` and export **per-image poses**:
   - `T_cw = [R|t]` (world→camera) and `T_wc = T_cw^{-1}` (camera→world)
   - Also save camera center `C = -R^T t`.
5. Show quick 2D/3D visualizations *in this notebook* and create `poses.csv` for downstream AR with PyTorch3D.

**Coordinate conventions**
- World frame is the checkerboard plane with Z=0, origin at the top‑left inner corner (standard OpenCV pattern points).
- OpenCV extrinsics follow `x_cam = R·X_world + t` ⇒ `[R|t]` is **world→camera**.
- We also export the inverse `T_wc` (camera pose in world).


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

In [ ]:
# !nvidia-smi -L || True
# !pip -q install opencv-python-headless numpy matplotlib pandas tqdm

import os, sys, json, glob, shutil, subprocess, textwrap, pprint
from pathlib import Path

# REPO_URL = "https://github.com/aissitt/CSE5283_calibration"
# REPO_DIR = Path("CSE5283_calibration")

# if not REPO_DIR.exists():
#     print("Cloning repo...", REPO_URL)
#     subprocess.run(["git", "clone", REPO_URL], check=True)
# else:
#     print("Repo already present — pulling latest")
#     subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=False)

# # Show tree
# print("\nRepo files:")
# subprocess.run(["bash", "-lc", "ls -R CSE5283_calibration | sed -n '1,120p'"])


In [ ]:
# === Config ===
# Put checkerboard images here (or change this path).
IMAGES_DIR = "data/images"  # default in the repo
OUT_DIR     = "data/results"

# Checkerboard definition: inner corners (OpenCV convention)
COLS = 9   # number of inner corners along width (columns)
ROWS = 6   # number of inner corners along height (rows)
SQUARE_SIZE_MM = 22.0  # physical size of a square in millimeters

# Preview image for undistortion demo (optional). If None, we'll pick the first image.
PREVIEW_IMAGE = None


In [ ]:
import os, glob, json, subprocess
from pathlib import Path

images = sorted(sum([glob.glob(os.path.join(IMAGES_DIR, f"*.{ext}")) for ext in ("jpg","jpeg","png","bmp")], []))
assert images, f"No images found under {IMAGES_DIR}"
if PREVIEW_IMAGE is None:
    PREVIEW_IMAGE = images[0]

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
corners_dir = os.path.join(OUT_DIR, "corners")
Path(corners_dir).mkdir(parents=True, exist_ok=True)

calib_json = os.path.join(OUT_DIR, "calibration.json")
undist_preview = os.path.join(OUT_DIR, "undistort_preview.jpg")

cmd = [
    sys.executable, "scripts/run_calibration.py",
    "--images_dir", IMAGES_DIR,
    "--cols", str(COLS), "--rows", str(ROWS),
    "--square_size", str(SQUARE_SIZE_MM),
    "--visualize_corners_to", corners_dir,
    "--out_json", calib_json,
    "--preview_image", PREVIEW_IMAGE,
    "--preview_out", undist_preview,
    "--undistort_alpha", "0.85",
    "--undistort_center_pp"
 ]
print("Running calibration:\n", " ".join(cmd))
res = subprocess.run(cmd)
if res.returncode != 0:
    raise RuntimeError(f"Calibration failed (exit {res.returncode}). Re-run cell for stderr output in the notebook kernel console.")

print("\nWrote:", calib_json)
if os.path.exists(undist_preview):
    from IPython.display import Image, display
    print("Undistortion preview:")
    display(Image(undist_preview))

In [ ]:
import os, subprocess, sys, glob
from IPython.display import Image, display

REPO_DIR = "CSE5283_calibration"
if not os.path.isdir(REPO_DIR):
    REPO_DIR = "."  # repo root
axes_dir = "data/results/axes"  # repo-relative to REPO_DIR
cmd = [
    sys.executable, "scripts/test_axes_overlay.py",
    "--output_dir", axes_dir,
    "--max_images", "9999"
 ]
print("Drawing axes overlays (cwd=", REPO_DIR, "):\n", " ".join(cmd))

# Run without capture first for transparency; capture only for silent mode
res = subprocess.run(cmd, cwd=REPO_DIR)
if res.returncode != 0:
    raise RuntimeError(f"axes overlay failed with code {res.returncode}")

# Show a small gallery (limit)
axes_imgs = sorted(glob.glob(os.path.join(REPO_DIR, axes_dir, "*.*")))[:6]
print(f"Showing {len(axes_imgs)} overlay(s):")
for p in axes_imgs:
    display(Image(p))

In [ ]:
import json, os, glob
import numpy as np
import pandas as pd
import cv2 as cv
from pathlib import Path
from IPython.display import display
import matplotlib.pyplot as plt

with open(os.path.join(OUT_DIR, "calibration.json"), "r") as f:
    calib = json.load(f)

# Attempt to read common keys from the repo's JSON structure
K = np.array(calib.get("K") or calib.get("camera_matrix"))
dist = np.array(calib.get("distCoeffs") or calib.get("dist_coeffs")).ravel()
rvecs = [np.array(v).ravel() for v in (calib.get("rvecs") or [])]
tvecs = [np.array(v).ravel() for v in (calib.get("tvecs") or [])]
per_img_rmse = calib.get("per_image_rmse") or calib.get("per_view_errors") or []
img_list = calib.get("image_paths") or calib.get("images") or sorted(sum([glob.glob(os.path.join(IMAGES_DIR, f"*.{ext}")) for ext in ("jpg","jpeg","png","bmp")], []))

assert K.size == 9, "Could not read 3x3 intrinsics K from calibration.json"
assert len(rvecs) == len(tvecs) == len(img_list), "rvecs/tvecs/image count mismatch"

def rt_to_Tcw(rvec, tvec):
    R, _ = cv.Rodrigues(rvec)
    Tcw = np.eye(4, dtype=float)
    Tcw[:3,:3] = R
    Tcw[:3, 3] = tvec.reshape(3)
    return Tcw

rows = []
Tcw_dict = {}
Twc_dict = {}
for i, (img_path, rvec, tvec) in enumerate(zip(img_list, rvecs, tvecs)):
    Tcw = rt_to_Tcw(rvec, tvec)
    Twc = np.linalg.inv(Tcw)
    R = Tcw[:3, :3]
    t = Tcw[:3, 3:4]
    C = (-R.T @ t).reshape(-1)  # camera center in world
    rmse = float(per_img_rmse[i]) if i < len(per_img_rmse) else np.nan

    rows.append({
        "image": img_path,
        "rmse": rmse,
        "C_x": C[0], "C_y": C[1], "C_z": C[2],
        "rvec_x": rvec[0], "rvec_y": rvec[1], "rvec_z": rvec[2],
        "t_x": tvec[0], "t_y": tvec[1], "t_z": tvec[2],
    })
    Tcw_dict[img_path] = Tcw.tolist()
    Twc_dict[img_path] = Twc.tolist()

poses_csv = os.path.join(OUT_DIR, "poses.csv")
import pandas as pd
df = pd.DataFrame(rows)
df.to_csv(poses_csv, index=False)
print("Saved:", poses_csv)
display(df.head())

poses_json = os.path.join(OUT_DIR, "poses_matrices.json")
with open(poses_json, "w") as f:
    json.dump({"K": K.tolist(), "distCoeffs": dist.tolist(), "T_cw": Tcw_dict, "T_wc": Twc_dict}, f, indent=2)
print("Saved:", poses_json)


In [ ]:
# Compute per-image RMSE (pixels) and write into poses.csv (robust version)
import os, glob, json
import numpy as np
import pandas as pd
import cv2 as cv
from pathlib import Path
from IPython.display import display

# Reuse notebook variables: IMAGES_DIR, OUT_DIR, COLS, ROWS, SQUARE_SIZE_MM
calib_path = os.path.join(OUT_DIR, "calibration.json")
poses_csv  = os.path.join(OUT_DIR, "poses.csv")

with open(calib_path, "r") as f:
    calib = json.load(f)

# --- Parse intrinsics/extrinsics robustly ---
K_raw = calib.get("K") or calib.get("camera_matrix")
if K_raw is None:
    raise ValueError("Could not find 'K' or 'camera_matrix' in calibration.json")
K = np.array(K_raw, dtype=np.float64).reshape(3,3)

# Distortion coefficients may come in several formats (list, nested list, None entries)
_dist_raw = calib.get("distCoeffs") or calib.get("dist_coeffs")
if _dist_raw is None:
    dist = np.zeros((5,), dtype=np.float64)  # assume no distortion if missing
else:
    # Flatten arbitrary nesting
    if isinstance(_dist_raw, dict):
        # Sometimes stored under a key like {'data': [...]} — try concatenating values
        cand = []
        for v in _dist_raw.values():
            if isinstance(v, (list, tuple)):
                cand.extend(v)
        _dist_raw = cand
    # Convert to 1D numeric list, filtering out None
    flat = []
    def _flatten(x):
        if isinstance(x, (list, tuple)):
            for y in x: _flatten(y)
        else: flat.append(x)
    _flatten(_dist_raw)
    flat = [x for x in flat if x is not None]
    try:
        dist = np.array(flat, dtype=np.float64).reshape(-1)
    except Exception as e:
        print("Warning: could not coerce distCoeffs, assuming zeros. Error:", e)
        dist = np.zeros((5,), dtype=np.float64)

# Fallback typical lengths (OpenCV: 4,5,8,12,14). If unexpected length, pad/truncate to 5.
if dist.size not in (4,5,8,12,14):
    if dist.size > 5:
        dist = dist[:5]
    else:
        dist = np.pad(dist, (0, max(0, 5 - dist.size)))

rvecs_raw = calib.get("rvecs") or []
tvecs_raw = calib.get("tvecs") or []

rvecs = [np.array(r, dtype=np.float64).reshape(3) for r in rvecs_raw]
tvecs = [np.array(t, dtype=np.float64).reshape(3) for t in tvecs_raw]

img_list = (calib.get("image_paths") or
            sorted(sum([glob.glob(os.path.join(IMAGES_DIR, f"*.{ext}")) for ext in ("jpg","jpeg","png","bmp")], [])))

assert len(img_list) == len(rvecs) == len(tvecs), (
    f"Mismatch: images({len(img_list)}) vs rvecs({len(rvecs)}) vs tvecs({len(tvecs)})")

# Build object points grid (OpenCV inner-corner convention)
objp = np.zeros((ROWS*COLS, 3), np.float32)
objp[:, :2] = np.mgrid[0:COLS, 0:ROWS].T.reshape(-1, 2)
objp *= (SQUARE_SIZE_MM / 1000.0)  # any consistent unit

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 1e-4)

per_image_rmse = []
for img_path, rvec, tvec in zip(img_list, rvecs, tvecs):
    gray = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    if gray is None:
        per_image_rmse.append(np.nan)
        continue
    found, corners = cv.findChessboardCorners(gray, (COLS, ROWS))
    if not found:
        per_image_rmse.append(np.nan)
        continue
    corners = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
    # Ensure correct shapes for projectPoints
    rvec_cv = rvec.reshape(3,1)
    tvec_cv = tvec.reshape(3,1)
    proj, _ = cv.projectPoints(objp, rvec_cv, tvec_cv, K, dist)
    proj = proj.reshape(-1, 2)
    corners = corners.reshape(-1, 2)
    err = np.sqrt(np.mean(np.sum((proj - corners)**2, axis=1)))  # pixels
    per_image_rmse.append(float(err))

# Update poses.csv
df = pd.read_csv(poses_csv)
if len(df) != len(per_image_rmse):
    print("Warning: row count mismatch; not updating rmse.")
else:
    if "rmse" not in df.columns:
        # keep 'image' first, insert after if needed
        if "image" in df.columns:
            insert_at = list(df.columns).index("image") + 1
        else:
            insert_at = 1
        df.insert(insert_at, "rmse", np.nan)
    df["rmse"] = per_image_rmse
    df.to_csv(poses_csv, index=False)
    print("Updated per-image rmse in:", poses_csv)
    display(df.head())

print("Distortion coeffs used:", dist.tolist())

In [ ]:
import numpy as np, json, os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused

with open(os.path.join(OUT_DIR, "poses_matrices.json"), "r") as f:
    P = json.load(f)

Twc_dict = P["T_wc"]

# Collect camera centers and rotations (camera -> world)
C_list, R_list = [], []
for _, Twc in Twc_dict.items():
    Twc = np.array(Twc, dtype=float)
    C_list.append(Twc[:3, 3])
    R_list.append(Twc[:3, :3])
C = np.array(C_list)

# Checkerboard outline (Z=0)
cols, rows, s = COLS, ROWS, SQUARE_SIZE_MM / 1000.0 
W = max((cols - 1) * s, 1e-6)
H = max((rows - 1) * s, 1e-6)
board_x = np.array([0, W, W, 0, 0])
board_y = np.array([0, 0, H, H, 0])
board_z = np.zeros_like(board_x)

def draw_triad_quiver(ax, origin, R, L, lw=2.0, alpha=0.95):
    """Draw RGB axis triad at origin with rotation R (camera/world axes as columns)."""
    o = origin.reshape(3)
    # X (red)
    ax.quiver(o[0], o[1], o[2], R[0,0]*L, R[1,0]*L, R[2,0]*L,
              color='r', arrow_length_ratio=0.15, linewidth=lw, alpha=alpha)
    # Y (green)
    ax.quiver(o[0], o[1], o[2], R[0,1]*L, R[1,1]*L, R[2,1]*L,
              color='g', arrow_length_ratio=0.15, linewidth=lw, alpha=alpha)
    # Z (blue)
    ax.quiver(o[0], o[1], o[2], R[0,2]*L, R[1,2]*L, R[2,2]*L,
              color='b', arrow_length_ratio=0.15, linewidth=lw, alpha=alpha)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')

# Board outline
ax.plot(board_x, board_y, board_z, 'k-', linewidth=2)

# Camera centers
if len(C) > 0:
    ax.scatter(C[:,0], C[:,1], C[:,2], s=20, c='k')

# --- auto scale for axis lengths (robust to mm vs m) ---
if len(C) > 0:
    span = np.max(np.ptp(C, axis=0))  # overall spread of camera positions
else:
    span = max(W, H)
span = float(span if span > 0 else max(W, H, 1.0))
Lc = 0.08 * span   # per-camera axis length
Lw = 0.12 * span   # world axis length
# -------------------------------------------------------

# World triad at chessboard origin (identity rotation)
draw_triad_quiver(ax, np.zeros(3), np.eye(3), Lw)

# Camera triads
for Ci, Ri in zip(C_list, R_list):
    draw_triad_quiver(ax, np.array(Ci), np.array(Ri), Lc)

# Labels / view / equal aspect
ax.set_xlabel("X_w")
ax.set_ylabel("Y_w")
ax.set_zlabel("Z_w")
ax.set_title("World + Camera RGB axes (checkerboard Z=0)")

pts = [np.zeros(3), [W,0,0], [0,H,0]] + list(C_list)
pts = np.array(pts)
mins, maxs = pts.min(axis=0), pts.max(axis=0)
max_range = (maxs - mins).max() or 1.0
center = (maxs + mins) / 2
ax.set_xlim(center[0] - max_range/2, center[0] + max_range/2)
ax.set_ylim(center[1] - max_range/2, center[1] + max_range/2)
ax.set_zlim(center[2] - max_range/2, center[2] + max_range/2)
ax.set_box_aspect((1,1,1))

ax.view_init(elev=30, azim=-60)
plt.tight_layout()
plt.show()
